In [1]:
# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

#personal modules
import acquire
import summarize as s
import prepare as p

# ignore warnings
import warnings
warnings.filterwarnings("ignore")



# Zillow


For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

#### Acquire and Summarize

1) Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [2]:
#acquiring data
df = pd.read_csv('zillow_all.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)


In [ ]:
s.summarize(df)


3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [22]:
def check_missing_values(df):
  
    cols = ['name', 'num_rows_missing', 'percent_rows_missing']
    df_1 = pd.DataFrame(columns=cols)
    
    for col in list(df.columns):
        num_rows_missing = df[col].isna().sum()
        percent_rows_missing = num_rows_missing / df.shape[0]
        df_2 = pd.DataFrame([{'name': col, 'num_rows_missing': num_rows_missing,
                               'percent_rows_missing': percent_rows_missing}])
        df_1 = pd.concat([df_1, df_2], axis=0)
    
    df_1.set_index('name', inplace=True)    
    return df_1

In [23]:
check_missing_values(df)

,num_rows_missing,percent_rows_missing
name,,
parcelid,0,0.00
typeconstructiontypeid,77356,1.00
storytypeid,77529,1.00
propertylandusetypeid,0,0.00
heatingorsystemtypeid,28008,0.36
...,...,...
buildingclassdesc,77564,1.00
heatingorsystemdesc,28008,0.36
propertylandusedesc,0,0.00


#### Prepare

1) Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [27]:
df.bedroomcnt.max()

16.0


2) Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

    - The input:
        - A dataframe
        - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
        - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
    - The output:
        - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
    - hint:
        - Look up the dropna documentation.
        - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
3) Encapsulate your work inside of functions in a wrangle_zillow.py module.

# Mall Customers

Notebook

1) Acquire data from the customers table in the mall_customers database.
2) Summarize the data (include distributions and descriptive statistics).
3) Detect outliers using IQR.
4) Split data into train, validate, and test.
5) Encode categorical columns using a one hot encoder (pd.get_dummies).
6) Handles missing values.
7) Scaling

Encapsulate your work in a wrangle_mall.py python module.
